## Introduction to `robotoc` 
# 1: Getting Started

## Installation (Ubuntu)
1. Please make sure that your OS is Ubuntu 22.04, 20.04, or 18.04
2. Install build tools via 
```
sudo apt install build-essential
```
3. Install `Eigen3` via
```
sudo apt install libeigen3-dev
```
4. Install [`Pinocchio`](https://github.com/stack-of-tasks/pinocchio) and its viewer ([`gepeto-viewer-corba`](https://github.com/Gepetto/gepetto-viewer-corba)) (see [here](https://stack-of-tasks.github.io/pinocchio/download.html) for detailed instruction)
  - Ubuntu 22.04:
```
sudo apt install -qqy lsb-release gnupg2 curl -y
echo "deb [arch=amd64] http://robotpkg.openrobots.org/packages/debian/pub $(lsb_release -cs) robotpkg" | sudo tee /etc/apt/sources.list.d/robotpkg.list
curl http://robotpkg.openrobots.org/packages/debian/robotpkg.key | sudo apt-key add -
sudo apt update -y
sudo apt install -qqy robotpkg-py310-pinocchio -y
echo export PATH=/opt/openrobots/bin:$PATH >> ~/.bashrc
echo export PKG_CONFIG_PATH=/opt/openrobots/lib/pkgconfig:$PKG_CONFIG_PATH >> ~/.bashrc
echo export LD_LIBRARY_PATH=/opt/openrobots/lib:$LD_LIBRARY_PATH >> ~/.bashrc
echo export PYTHONPATH=/opt/openrobots/lib/python3.10/site-packages:$PYTHONPATH >> ~/.bashrc
echo export CMAKE_PREFIX_PATH=/opt/openrobots:$CMAKE_PREFIX_PATH >> ~/.bashrc
sudo apt update && sudo apt install robotpkg-py310-qt5-gepetto-viewer-corba -y
```
  - Ubuntu 20.04:
```
sudo apt install -qqy lsb-release gnupg2 curl -y
echo "deb [arch=amd64] http://robotpkg.openrobots.org/packages/debian/pub $(lsb_release -cs) robotpkg" | sudo tee /etc/apt/sources.list.d/robotpkg.list
curl http://robotpkg.openrobots.org/packages/debian/robotpkg.key | sudo apt-key add -
sudo apt update -y
sudo apt install -qqy robotpkg-py38-pinocchio -y
echo export PATH=/opt/openrobots/bin:$PATH >> ~/.bashrc
echo export PKG_CONFIG_PATH=/opt/openrobots/lib/pkgconfig:$PKG_CONFIG_PATH >> ~/.bashrc
echo export LD_LIBRARY_PATH=/opt/openrobots/lib:$LD_LIBRARY_PATH >> ~/.bashrc
echo export PYTHONPATH=/opt/openrobots/lib/python3.8/site-packages:$PYTHONPATH >> ~/.bashrc
echo export CMAKE_PREFIX_PATH=/opt/openrobots:$CMAKE_PREFIX_PATH >> ~/.bashrc
sudo apt update && sudo apt install robotpkg-py38-qt5-gepetto-viewer-corba -y
```
5. Install `robotoc` via
```
git clone https://github.com/mayataka/robotoc
cd robotoc
mkdir build && cd build
cmake .. -DCMAKE_BUILD_TYPE=Release -DOPTIMIZE_FOR_NATIVE=ON
sudo make install -j$(nproc)
```
  and set Python PATH 
  - Ubuntu 22.04:
```
echo export PYTHONPATH=/usr/local/lib/python3.10/site-packages:$PYTHONPATH >> ~/.bashrc
```
  - Ubuntu 20.04:
```
echo export PYTHONPATH=/usr/local/lib/python3.8/site-packages:$PYTHONPATH >> ~/.bashrc
```
6. Install `pybullet` and `meshcat` via
```
pip3 install pybullet meshcat
```

## Installation (Mac)
The following instructions are not well-tested.   
Therefore, I'd like to recommend using Ubuntu.   
1. Install `Pinocchio` 
```
brew tap gepetto/homebrew-gepetto
brew install pinocchio
```
2. Check the full-path to `g++`. (this is necessary because the default compiler is clang in Mac)
```
ls -l /usr/local/bin | grep g++
```
3. Install `robotoc` via
```
git clone https://github.com/mayataka/robotoc
cd robotoc
mkdir build && cd build
cmake .. -DCMAKE_BUILD_TYPE=Release -DOPTIMIZE_FOR_NATIVE=ON -DCMAKE_CXX_COMPILER=FULL_PATH_TO_GPLUSPLUS
sudo make install -j$(nproc)
```
  and set Python PATH 
```
echo export PYTHONPATH=/usr/local/lib/python3.9/site-packages:$PYTHONPATH >> ~/.zshrc
```
4. Install `pybullet` and `meshcat` via
```
pip3 install pybullet meshcat
```

## Test Whole-body MPC
To verify the installation was successful, we run a whole-body MPC simulation.   
The other notebooks instruct the details of `robotoc` API.

### Import modules

In [ ]:
import robotoc
import numpy as np
from a1_simulator import A1Simulator

### Create robot

In [ ]:
path_to_urdf = 'urdf/a1_description/urdf/a1.urdf'
contact_frames = ['FL_foot', 'RL_foot', 'FR_foot', 'RR_foot'] 
contact_types = [robotoc.ContactType.PointContact for i in contact_frames]
baumgarte_time_step = 0.05
robot = robotoc.Robot(path_to_urdf, robotoc.BaseJointType.FloatingBase, 
                      contact_frames, contact_types, baumgarte_time_step)

### Create whole-body MPC

In [ ]:
T = 0.5
N = 18
max_steps = 3
nthreads = 4
mpc = robotoc.MPCFlyingTrot(robot, T, N, max_steps, nthreads)

### Commands sent to whole-body MPC (`vcom_cmd`, `yaw_rate_cmd`)

In [ ]:
# cmd_type = 'forward'
# cmd_type = 'backward'
# cmd_type = 'side'
# cmd_type = 'curve'
cmd_type = 'rotation'

if cmd_type == 'forward':
    step_length = np.array([0.30, 0.0, 0.0]) 
    yaw_cmd = 0.
elif cmd_type == 'backward':
    step_length = np.array([-0.1, 0.0, 0.0]) 
    yaw_cmd = 0.
elif cmd_type == 'side':
    step_length = np.array([0.0, 0.15, 0.0]) 
    yaw_cmd = 0.
elif cmd_type == 'curve':
    step_length = np.array([0.1, 0.0, 0.0]) 
    yaw_cmd = np.pi / 12.
elif cmd_type == 'rotation':
    step_length = np.array([0.0, 0.0, 0.0]) 
    yaw_cmd = np.pi / 6.
    
step_height = 0.1
stance_time = 0.15
flying_time = 0.1
swing_start_time = 0.5

swing_time = stance_time + flying_time
vcom_cmd = step_length / swing_time
yaw_rate_cmd = yaw_cmd / swing_time

### Create a foot step planner

In [ ]:
planner = robotoc.FlyingTrotFootStepPlanner(robot)
raibert_gain = 0.2
planner.set_gait_pattern(vcom_cmd, yaw_rate_cmd, swing_time, stance_time, raibert_gain)
mpc.set_gait_pattern(planner, step_height, flying_time, stance_time, swing_start_time)

### Create initial state (`q`, `v`) and initial time (`t`)

In [ ]:
q = np.array([0, 0, 0.3181, 0, 0, 0, 1, 
              0.0,  0.67, -1.3, 
              0.0,  0.67, -1.3, 
              0.0,  0.67, -1.3, 
              0.0,  0.67, -1.3])
v = np.zeros(robot.dimv())
t = 0.0

### Initialize MPC (performs few Newton-type iterations)

In [ ]:
option_init = robotoc.SolverOptions()
option_init.max_iter = 10
mpc.init(t, q, v, option_init)

### Set solver options to MPC (number of the Newton-type iterations at each sampling time)

In [ ]:
option_mpc = robotoc.SolverOptions()
option_mpc.max_iter = 2 # MPC iterations
mpc.set_solver_options(option_mpc)

### Create a simulator and run the simulation!

In [ ]:
sim_time_step = 0.0025 # 400 Hz MPC
sim_start_time = 0.0
sim_end_time = 10.0

sim = A1Simulator(path_to_urdf, sim_time_step, sim_start_time, sim_end_time)

sim.set_camera(2.0, 45, -10, q[0:3]+np.array([0.1, 0.5, 0.]))
sim.run_simulation(mpc, q, v, feedback_delay=True, verbose=False, record=False)